In [1]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!cp /content/drive/MyDrive/EEE\ 485\ Project/Code/Sunum\ kodu/metrics.ipynb /content

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
import import_ipynb
import itertools
from metrics import (r2,
                    mse,
                    mae,
                    mape,
)

importing Jupyter notebook from metrics.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/EEE 485 Project/Code/main.ipynb': No such file or directory


In [7]:

class Scaling():

  """
  Applies Standard Scaler and Min Max Scaler.
  ;minmax: when True runs Min Max Scaler
  """
  def __init__(self):
    self.__means = {}
    self.__vars  = {}
    self.__mins = {}
    self.__maxs = {}
  def fit(self, dffff, minmax = None):
      if minmax:
        for column in dffff.columns:
          min_one =(dffff[column]).min(axis=0)
          max_one = (dffff[column]).max(axis=0)
          final_col_train = (dffff[column] - min_one)/(max_one-min_one)
          dffff[column] = final_col_train
          self.__mins[column] = min_one
          self.__maxs[column] = max_one
      else:
        for column in dffff.columns:
          mean_one = np.mean(np.array(dffff[column]), 0)
          final_col_train = (dffff[column] - mean_one)
          stdd = np.std(final_col_train, ddof=0)
          final_col_train =  final_col_train/stdd

          dffff[column] = final_col_train

          self.__means[column] = mean_one
          self.__vars[column] = stdd
      return dffff

  def transform(self, df, minmax = None):
      if minmax:
        for column in df.columns:
          final_col_test = ( df[column].values - self.__mins[column])/(self.__maxs[column] - self.__mins[column]) 
          df[column] = final_col_test
      else:
        for column in df.columns:
          final_col_test = ( df[column].values - self.__means[column])/self.__vars[column] 
          df[column] = final_col_test
      
      return df


In [8]:
class PCA():

  """
  Applies PCA to train and test set.
  Draws PVE plot to chose suitable percentage of PC to pick.
  """
  def __init__(self):
    self.__eigenvectors = []
    self.__pve_list = []
    self.__pca = 0
    self.__number_of_components = 0
  def get_PCA_plot(self, df, threshold):
    df = self.__pve_list
    self.__number_of_components = [x+1 for x in range(len(df))]
    return self.__number_of_components, self.__pve_list

  def get_PVE(self):
    for i in range(len(self.__pve_list)):
      print(f"Number of components:{self.__number_of_components[i]} -> PVE:{self.__pve_list[i]}")
    
  def preprocess_train(self, X_train, threshold):
    
    df_meaned = (X_train - np.mean(X_train, axis = 0))
    cov_matrix = np.cov(df_meaned, rowvar = False)
    values, vectors = np.linalg.eigh(cov_matrix)

    sorted_index = np.argsort(values)[::-1]
    sorted_eigenvalue = values[sorted_index]
    sorted_eigenvectors = vectors[:,sorted_index]

    pve = []
    for i in range(len(sorted_eigenvalue)):
        pve_percen = sum(sorted_eigenvalue[:i+1])*100 / np.sum(sorted_eigenvalue)
        pve.append(pve_percen)
    
    self.__pve_list = pve
    threshold_list = [pve.index(i) for i in pve if i >= threshold]
    threshold_out = threshold_list[0]

    sorted_index = sorted_index[:threshold_out+1]
    sorted_eigenvalue = sorted_eigenvalue[:threshold_out+1]
    sorted_eigenvectors = sorted_eigenvectors[:,:threshold_out+1]
    self.__eigenvectors = sorted_eigenvectors

    df_PCA = np.dot(sorted_eigenvectors.transpose(),df_meaned.transpose()).transpose()
    df_PCA = pd.DataFrame(data=df_PCA,columns=[f"principal_componenet {y+1}" for y in range(df_PCA.shape[1])])  
    
    return df_PCA
  
  def preprocess_test(self,X_test,X_train):
   
    df_meaned = X_test - np.mean(X_train, axis = 0)
    df_PCA = np.dot(self.__eigenvectors.transpose(),df_meaned.transpose()).transpose()
    df_PCA = pd.DataFrame(data=df_PCA,columns=[f"principal_componenet {y+1}" for y in range(df_PCA.shape[1])])  

    return df_PCA


In [9]:
class Linear_Regression():
 
  def __init__(self, path):

    self.__path = path
    self._X_train = pd.DataFrame([])
    self._Y_train = pd.DataFrame([])
    self._X_test = pd.DataFrame([])
    self._Y_test = pd.DataFrame([])
    self._X_val = pd.DataFrame([])
    self._Y_val = pd.DataFrame([])



  def train_test(self, scaler_is = None, pca_is = None):
      """
      ;scaler_is: when True performs Standard Scaler
      ;pca_is: when True performs PCA
      """
      df_original = pd.read_csv(self.__path, parse_dates = True, index_col = 0) 
      trainn = df_original.loc[:"2017-11-30 23:00:00"]
      val = df_original.loc["2017-12-01 00:00:00":"2017-12-31 23:00:00"]
      test = df_original.loc["2018-01-01 00:00:00":]
      scaler = Scaling()

      if scaler_is:
        trainn = scaler.fit(trainn, minmax = False)
        test = scaler.transform(test, minmax = False)
        val = scaler.transform(val, minmax = False)
      else: 
        trainn = scaler.fit(trainn, minmax = True)
        test = scaler.transform(test, minmax = True)
        val = scaler.transform(val, minmax = True)
      


      self.__Y_train = pd.DataFrame(trainn.iloc[:, -1])
      self.__Y_test = pd.DataFrame(test.iloc[:, -1])
      self.__Y_val = pd.DataFrame(val.iloc[:, -1])

      self.__X_train = pd.DataFrame(trainn.iloc[:, :-1])
      self.__X_test = pd.DataFrame(test.iloc[:, :-1])
      self.__X_val = pd.DataFrame(val.iloc[:, :-1])
      if pca_is :
        pca_comp = PCA()
        self.__X_train = pca_comp.preprocess_train(np.array(self.__X_train),95)
        self.__X_test = pca_comp.preprocess_test(np.array(self.__X_test), np.array(self.__X_train))
        self.__X_val = pca_comp.preprocess_test(np.array(self.__X_val), np.array(self.__X_train))

      return self.__Y_train,self.__Y_val, self.__Y_test,self.__X_train,self.__X_val,self.__X_test

  def loss_function(self,features):

    RSS = 0
    error = np.sum((np.sum(self.__Y_train ,1)-np.sum((self.__X_train)*features[:-1]+ features[-1],1))**2)
    RSS = error/(2*(self.__X_train).shape[0])
    return RSS 


  def gradient_descent(self, L,h,k,features, lasso = None, ridge = None, elasticnet =None):
 
    grad = np.zeros(len(features))
    for i in range(len(features[:-1])):
      if lasso:
        if features[i] > 0:
          grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*self.__X_train.iloc[:,i] + h)
        else:
          grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*self.__X_train.iloc[:,i] - h)
      elif ridge:
          grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*self.__X_train.iloc[:,i] + 2*h*features[i])
      elif elasticnet:
          if features[i] > 0:
            grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*self.__X_train.iloc[:,i] + h + k*features[i])
          else:
            grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*self.__X_train.iloc[:,i] - h + k*features[i])
      else:
          grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*(self.__X_train).iloc[:,i])

    grad[-1] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1)))
    features -= L*grad
    return features

  # def gradient_descent(self,L,features):

  #   grad = np.zeros(len(features))
  #   for i in range(len(features[:-1])):
  #     grad[i] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1))*(self.__X_train).iloc[:,i])

  #   grad[-1] = -(2/len(self.__Y_train))*np.sum((np.sum(self.__Y_train,1)-np.sum(self.__X_train*features[:-1]+ features[-1],1)))
  #   features -= L*grad
  #   return features


  def training_testing(self, LR, H, K, epoch):
    """
    trains one model
    """
    xlist = []
    ylist = [] 
    features =  np.ones(self.__X_train.shape[1]+1)/5
    for i in range(epoch):
        features = Linear_Regression.gradient_descent(self, LR,H,K,features, lasso = False, ridge = False, elasticnet = True)
        xlist.append(i)
        ylist.append(Linear_Regression.loss_function(self,features)) 
    y_pred = np.sum(self.__X_test*features[:-1]+ features[-1],1)
    y_pred_train = np.sum(self.__X_train*features[:-1]+features[-1],1) 
    print("MSE", mse(self.__Y_test, y_pred),"MAE", mae(self.__Y_test, y_pred),"R2 score", r2(self.__Y_test, y_pred))
    print("Train MSE", mse(self.__Y_train, y_pred_train),"MAE", mae(self.__Y_train, y_pred_train),"R2 score", r2(self.__Y_train, y_pred_train))

    plt.figure(figsize=(10,10))
    plt.subplot(211)   
    plt.plot(xlist,ylist)
    plt.title("Epoch vs Training Loss")
    plt.subplot(212)
    plt.plot(self.__Y_test.index, self.__Y_test.values)
    plt.plot(y_pred.index, y_pred.values)
    plt.title("Predictions and Ground Truth Test")
    plt.legend(["test","pred"])
    return y_pred, y_pred_train

  def train_val(self, LR, H, K, epochh):
    
      """
      ;LR: learning rate
      ;H: L1 penalty
      ;K: L2 penalty
      ;epochh: epoch number
      validation function 
      """

      xlist = []
      ylist = []
      features =  np.ones(self.__X_train.shape[1]+1)/5
      for i in range(epochh):
          features = Linear_Regression.gradient_descent(self, LR,H,K, features, lasso = False, ridge = True, elasticnet = False)
          xlist.append(i)
          ylist.append(Linear_Regression.loss_function(self,features)) 
      y_pred = np.sum(self.__X_val*features[:-1]+ features[-1],1)
      plt.plot(xlist,ylist)
      plt.title("Epoch vs Training Loss")
     
      return  y_pred

  def search(self, parameters):
    """
    takes 1 hour with 15 paramaters.
    parameter search for learning rate, lambdas, and epoch
    """
    param_values = [v for v in parameters.values()] 
    error_of_choice= []
    whole_turns = []
    for lr,h,k, epoch in itertools.product(*param_values):
        y_pred_val = Linear_Regression.train_val(self, lr, h,k, epoch)
        MSE = mse(self.__Y_val,y_pred_val)
        error_of_choice.append(MSE)
        whole_turns.append([lr,epoch,h]) 
        print('learning rate: %.4f, epoch: %.4f, lambda: %.4f, MSE:%.3f' % (lr,epoch,h,MSE))
    min_error_index =error_of_choice.index(min(error_of_choice))
    choice = whole_turns[min_error_index]
    print('Best parametres','learning rate:',choice[0],'epoch:',choice[1],'lambda:',choice[2], 'MSE:',min(error_of_choice)) 

    return choice